# <span style="font-width:bold; font-size: 3rem; color:#1EB182;">**Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Feature Pipeline</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/nyc_taxi_fares/2_feature_pipeline.ipynb)

## 🗒️ This notebook is divided into 2 sections:
1. Data generation.
2. Insert new data into the Feature Store.

### <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import warnings

# Mute warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
from datetime import datetime
import time 
import os 

from features import nyc_taxi_rides, nyc_taxi_fares

___

## <span style="color:#ff5f27;"> 🪄 Generating new data</span>

### <span style='color:#ff5f27'> 🚖 Rides Data

In [ ]:
df_rides = nyc_taxi_rides.generate_rides_data(150)

df_rides

In [ ]:
df_rides = nyc_taxi_rides.calculate_distance_features(df_rides)

In [ ]:
df_rides = nyc_taxi_rides.calculate_datetime_features(df_rides)

In [ ]:
# now save the newly-generated ride_ids.
# it will be retrieved and used in for fares data generation
ride_ids = df_rides.ride_id

In [ ]:
for col in ["passenger_count", "taxi_id", "driver_id"]:
    df_rides[col] = df_rides[col].astype("int64")

### <span style='color:#ff5f27'> 💸 Fares Data

In [ ]:
df_fares = nyc_taxi_fares.generate_fares_data(150)

df_fares

In [ ]:
df_fares = df_fares.astype("int64")

In [ ]:
# lets load our ride_ids which were created moments ago for rides_fg
df_fares["ride_id"] = ride_ids

In [ ]:
for col in ["tolls", "total_fare"]:
    df_fares[col] = df_fares[col].astype("double")

___

## <span style="color:#ff5f27;"> 📡 Connecting to the Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()
fs = project.get_feature_store() 

In [ ]:
rides_fg = fs.get_or_create_feature_group(name="nyc_taxi_rides",
                                          version=1)   

fares_fg = fs.get_or_create_feature_group(name="nyc_taxi_fares",
                                          version=1)   

---

## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
rides_fg.insert(df_rides)

In [ ]:
fares_fg.insert(df_fares)

---

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03 </span>

In the next notebook, you will create a feature view, training dataset, train a model and save it to Hopsworks Model Registry.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/nyc_taxi_fares/3_training_dataset_and_modeling.ipynb)